## Import data

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/eth_CS4NLP/project/output'

Mounted at /content/drive
label_diff.csv
label_diff.gsheet
new_test_pred_dbmdz_bert-base-italian-xxl-cased.csv
new_test_pred_dbmdz_bert-base-italian-xxl-cased.csv.gsheet
pred_bert_dbmdz_bert-base-italian-cased.csv
pred_bert_dbmdz_bert-base-italian-uncased.csv
pred_bert_dbmdz_bert-base-italian-xxl-cased.csv
pred_bert_m-polignano-uniba_bert_uncased_L-12_H-768_A-12_italian_alb3rt0.csv
pred_bert_m-polignano-uniba:bert_uncased_L-12_H-768_A-12_italian_alb3rt0.csv
pred_bert_mrm8488_bert-italian-finedtuned-squadv1-it-alfa.csv
pred_giacomo.txt


In [3]:
#test prediction
pred_test_lr_path = '/content/drive/MyDrive/eth_CS4NLP/project/output/pred_giacomo.txt'
pred_test_bertxxl_path = '/content/drive/MyDrive/eth_CS4NLP/project/output/new_test_pred_dbmdz_bert-base-italian-xxl-cased.csv'

In [4]:
pred1 = '/content/drive/MyDrive/eth_CS4NLP/project/output/pred_bert_m-polignano-uniba_bert_uncased_L-12_H-768_A-12_italian_alb3rt0.csv'
pred2 = '/content/drive/MyDrive/eth_CS4NLP/project/output/pred_bert_dbmdz_bert-base-italian-cased.csv'

In [5]:
dial_label = {
    0 : 'EML',
    1 : 'NAP',
    2 : 'PMS',
    3 : 'FUR',
    4 : 'LLD',
    5 : 'LIJ',
    6 : 'LMO',
    7 : 'ROA_TARA',
    8 : 'SCN', 
    9 : 'VEC',
    10 : 'SC'
}

fold_label = {
    'EML' : 0,
    'NAP' : 1,
    'PMS' : 2,
    'FUR' : 3,
    'LLD' : 4,
    'LIJ' : 5,
    'LMO' : 6,
    'ROA_TARA' : 7,
    'SCN' : 8,
    'VEC' : 9,
    'SC' : 10
}

def explain_label(label : int) -> str:
    """ 
    Given an integer label, convert it to the corresponding string label
    :param int label: integer label to be converted
    :return: string corresponding to the given label
    """
    return dial_label[label]

def encode_label(label : str) -> int:
    """ 
    Given a string label, encode it to the corresponding index
    :param string label: string label to be converted
    :return: int corresponding to the given label
    """
    return fold_label[label]

# Compare prediction on testset

In [6]:
df_test_lr = pd.read_csv(pred_test_lr_path, sep = "\t", names=["label_lr", "text"])
df_test_bert= pd.read_csv(pred_test_bertxxl_path, sep = "\t")
df_test_bert['pred_label']=df_test_bert['pred_label'].apply(explain_label)
df_test_label = df_test_lr.join(df_test_bert['pred_label'])
df_test_label = df_test_label[['pred_label','label_lr','text']]
df_test_label.columns = ['lable_bert','label_lr','text']


In [7]:
pd.set_option('display.max_colwidth', None)
label_diff = df_test_label.query('label_lr != lable_bert')
label_diff.head(4)

,lable_bert,label_lr,text
5,NAP,ROA_TARA,"Vine paisane, e vine mbuttigliate col sùvero d' argento e l' etichetta, liquori delle frabbiche premiate, curassò, strega, cúmmel e anisetta:"
6,NAP,LLD,In quella provinzia è zità e chastelle asai;
9,LIJ,SCN,dumandò u Pinocchiu cun anscêtæ e affannu.
10,LLD,LMO,"un omeneto da ride, ve digo, outo poco pi de doi forminantes."


In [8]:
from google.colab import files
label_diff.to_csv('label_diff.csv',index=False)
files.download('label_diff.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
df_test_lr.shape

(11087, 2)

In [11]:
df_test_bert.shape

(11087, 1)

## Ensembling
- [x] Hard label: Every vote counts as a single vote.
- [x] Hard labels with Weighted: Models with better performance on the validation set get more influence.
- [x] Soft Label Election with Logits.
- [x] Soft Label Election with softmaxed Logits (probs).


## Hard Label

In [9]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
qa = VotingClassifier(estimators=[('uniba', pred1),('dbmdz', pred1)], voting='soft')
print(qa)

VotingClassifier(estimators=[('uniba',
                              '/content/drive/MyDrive/eth_CS4NLP/project/output/pred_bert_m-polignano-uniba_bert_uncased_L-12_H-768_A-12_italian_alb3rt0.csv'),
                             ('dbmdz',
                              '/content/drive/MyDrive/eth_CS4NLP/project/output/pred_bert_m-polignano-uniba_bert_uncased_L-12_H-768_A-12_italian_alb3rt0.csv')],
                 voting='soft')
